# M3 month

In [1]:
import numpy as np
import pyramid
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from pyramid.arima import auto_arima
from sklearn import metrics
import tsfresh

## 1 TS with exogenous (Tsfresh)

In [3]:
df = pd.read_excel('M3C.xls',sheet_name='M3Month',index_col='Series')
df = df[df.Category == 'MICRO       ']
df

,N,NF,Category,Starting Year,Starting Month,1,2,3,4,5,...,135,136,137,138,139,140,141,142,143,144
Series,,,,,,,,,,,,,,,,,,,,,
N1402,68,18,MICRO,1990,1,2640.0,2640.0,2160.0,4200.0,3360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1403,68,18,MICRO,1990,1,1680.0,1920.0,120.0,1080.0,840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1404,68,18,MICRO,1990,1,1140.0,720.0,4860.0,1200.0,3150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1405,68,18,MICRO,1990,1,180.0,940.0,2040.0,800.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1406,68,18,MICRO,1990,1,2000.0,1550.0,4450.0,3050.0,3050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1407,68,18,MICRO,1990,1,1200.0,2850.0,1350.0,1500.0,1950.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1408,68,18,MICRO,1990,1,1350.0,2400.0,1950.0,2250.0,3000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1409,68,18,MICRO,1990,1,2700.0,3600.0,1950.0,4650.0,3900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N1410,68,18,MICRO,1990,1,3680.0,2600.0,2460.0,3860.0,4640.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
N_series = 9
N_series_1990 = 20
N = df['N'][N_series]
NF = df['NF'][N_series]
start_year = df['Starting Year'][N_series]
start_month = df['Starting Month'][N_series]
indices_of_exogenous = np.arange(10,20)
K1 = N-13
K2 = N+5
train_data = np.array(df.iloc[N_series,5:K1]).astype(np.float64)
validate_data = np.array(df.iloc[N_series,K1:K2]).astype(np.float64)

In [89]:
datelist = pd.date_range('1990-1',periods=68, dtype='period[M]', freq='MS')
TS = pd.Series(data=np.concatenate([train_data,validate_data]), index=datelist)
TS.head()
TS_train = TS[:50]
TS_validate = TS[50:]

#### Tsfresh

In [104]:
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute

df_shift, y = make_forecasting_frame(TS_train.T, kind="price", max_timeshift=1, rolling_direction=1)
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, 
                     show_warnings=False)

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:394: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
Feature Extraction: 100%|██████████| 10/10 [00:01<00:00,  5.00it/s]
 

In [106]:
for c in X.columns:
    print (c)

value__abs_energy
value__absolute_sum_of_changes
value__agg_autocorrelation__f_agg_"mean"
value__agg_autocorrelation__f_agg_"median"
value__agg_autocorrelation__f_agg_"var"
value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"
value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"
value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"
value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"
value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"
value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"
value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"
value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"
value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"
value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"rvalue"
value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"slope"
value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"stderr"
value__agg_linear_trend__f_agg_"m

In [98]:
X_exog = ((X.T)[:200]).T
X_exog.shape

(49, 200)

#### Training

In [99]:
stepwise_fit = auto_arima(y, exogenous= X_exog, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, seasonal=True, d=1, max_d =3, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=0.736 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=nan, BIC=nan, Fit time=0.360 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 0, 12); AIC=nan, BIC=nan, Fit time=1.523 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=0.759 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=0.756 seconds
Fit ARIMA: order=(1, 1, 1) sea

/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   49
Model:             SARIMAX(1, 1, 0)x(1, 1, 0, 12)   Log Likelihood                     nan
Date:                            Wed, 04 Jul 2018   AIC                                nan
Time:                                    16:54:33   BIC                                nan
Sample:                                         0   HQIC                               nan
                                             - 49                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -2.124e-08        nan        nan        nan         nan         nan
x1         -2.265e-05   6.13e-15   -3.7e+09      0.000   -2.27e-05   -2.27e-05
x2             0.0020      8e-15   2.51e+11      0.000       0.002       0.002
x3            -0.0002   1.39e-15  -1.61e+11      0.000      -0.000      -0.000
x4            -0.0002        nan        nan        nan         nan         nan
x5          4.021e-05        nan        nan        nan         nan         nan
x6            -0.1683   3.99e-15  -4.22e+13      0.000      -0.168      -0.168
x7             0.0001        nan        nan        nan         nan         nan
x8             0.1912   6.48e-15   2.95e+13      0.000       0.191       0.191
const      -7.737e-13        nan        nan        nan         nan         nan
x9         -1.993e-12        nan        nan        nan         nan         nan
x10         2.073e-12        nan        nan        nan         nan         nan
x11         1.572e-13        nan        nan        nan         nan         nan
x12         1.126e-13        nan        nan        nan         nan         nan
x13           -0.1015   3.68e-15  -2.76e+13      0.000      -0.101      -0.101
x14           -0.0004        nan        nan        nan         nan         nan
x15           -0.0357        nan        nan        nan         nan         nan
x16           -0.1952   4.15e-15   -4.7e+13      0.000      -0.195      -0.195
x17           -0.0623    1.3e-15  -4.79e+13      0.000      -0.062      -0.062
x18            0.0004        nan        nan        nan         nan         nan
x19            0.0492   5.57e-15   8.82e+12      0.000       0.049       0.049
x20         8.724e-14        nan        nan        nan         nan         nan
x21         3.057e-13        nan        nan        nan         nan         nan
x22        -7.859e-13        nan        nan        nan         nan         nan
x23         6.319e-12        nan        nan        nan         nan         nan
x24        -5.543e-12        nan        nan        nan         nan         nan
x25           -0.0238   3.63e-15  -6.55e+12      0.000      -0.024      -0.024
x26            0.0001        nan        nan        nan         nan         nan
x27           -0.0661        nan        nan        nan         nan         nan
x28            0.0445   1.27e-15   3.51e+13      0.000       0.044       0.044
x29            0.0297   4.47e-15   6.65e+12      0.000       0.030       0.030
x30         9.845e-05        nan        nan        nan         nan         nan
x31           -0.0962   3.05e-15  -3.16e+13      0.000      -0.096      -0.096
x32         4.871e-17        nan        nan        nan         nan         nan
x33        -1.402e-17        nan        nan        nan         nan         nan
x34         3.267e-18        nan        nan        nan         nan         nan
x35                 0        nan        nan        n

In [95]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd
from bokeh.models import Span
# init bokeh
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    vline = Span(location=49,dimension='height', line_color='red',line_width=1)
    p.renderers.extend([vline])
    
    return p

Loading BokehJS ...

In [96]:
in_sample_preds = stepwise_fit.predict_in_sample(exogenous=X_exog)
print (metrics.mean_absolute_error(y, in_sample_preds))

1417.8819068855696


In [97]:
show(plot_arima(train_data, in_sample_preds, 
                title="Original Series & In-sample Predictions", forecast_start=0))

#### Easy approach

In [213]:
next_validate = stepwise_fit.predict(n_periods=18,exogenous=exogenous_data_2)
full_data = np.concatenate([train_data,validate_data])
full_pred = np.concatenate([in_sample_preds,next_validate])
show(plot_arima(full_data, full_pred,forecast_start=0,title="Original Series & Predictions"))

#### prediction of exogenous

In [214]:
def predict_exogenous(s):
    train_data_ex = np.array(df.iloc[s,5:K1]).astype(np.float64)
    stepwise_fit_ex = auto_arima(train_data_ex, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, seasonal=True, d=1, max_d =3, D=1, trace=False,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise
    return stepwise_fit_ex.predict(n_periods=18)

In [215]:
exogenous_data_2_predicted = []
for s in indices_of_exogenous:
    exogenous_data_2_predicted.append(predict_exogenous(s))
exogenous_data_2_predicted = np.array(exogenous_data_2_predicted).T

In [216]:
# for i in range(10,20):
#     show(plot_arima((exogenous_data_2.T)[i], (exogenous_data_2_predicted.T)[i],forecast_start=0))

In [217]:
next_validate_pred = stepwise_fit.predict(n_periods=18,exogenous=exogenous_data_2_predicted)
full_pred_ex = np.concatenate([in_sample_preds,next_validate_pred])
show(plot_arima(full_data, full_pred_ex,forecast_start=0,title="Original Series & Predictions"))
show(plot_arima(full_data, full_pred,forecast_start=0,title="Original Series & Predictions (easy)"))

In [219]:
print ('Easy MAE:',metrics.mean_absolute_error(validate_data,next_validate))
print ('New MAE:',metrics.mean_absolute_error(validate_data,next_validate_pred))

Easy MAE: 2018.134996194367
New MAE: 1908.5921064578583
